# 1. Import Libraries 

In [1]:
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from rdflib.namespace import OWL, RDF, RDFS, FOAF, XSD
from rdflib.util import guess_format

from SPARQLWrapper import SPARQLWrapper, JSON, N3

import pandas as pd
from pandas.api.types import is_numeric_dtype
pd.set_option('display.max_columns', None)

#Used from INM713 Semantic Web Technologies and Knowledge Graphs Lab 5
from isub import isub
from lookup import DBpediaLookup

import csv
import owlrl

import math

C:\Users\Abhimanyu Acharya\anaconda3\envs\diss_kg\lib\site-packages\rdflib_jsonld\__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,


# 2. Import Data

### 2.1 Import animal axonomy data

In [2]:
taxonomy_df = pd.read_csv("C:/Dissertation/Data/Mammal Taxonomy/Mammal Taxonomy.csv")
taxonomy_df

,sciName,animal_label,animal_class,mainCommonName,otherCommonNames,class,infraclass,magnorder,superorder,order,suborder,infraorder,parvorder,superfamily,family,subfamily,tribe,genus,subgenus,biogeographicRealm,CMW_sciName
0,Antilope_cervicapra,1,antelope,Blackbuck,Indian Antelope,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,ARTIODACTYLA,RUMINANTIA,PECORA,NaN,NaN,BOVIDAE,ANTILOPINAE,ANTILOPINI,Antilope,NaN,Palearctic,Antilope_cervicapra
1,Pantholops_hodgsonii,1,antelope,Chiru,Tibetan Antelope,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,ARTIODACTYLA,RUMINANTIA,PECORA,NaN,NaN,BOVIDAE,ANTILOPINAE,CAPRINI,Pantholops,NaN,Palearctic,Pantholops_hodgsonii
2,Ursus_arctos,2,grizzly+bear,Brown Bear,Grizzly Bear|Kodiak Bear,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,CARNIVORA,CANIFORMIA,ARCTOIDEA,NaN,URSOIDEA,URSIDAE,URSINAE,NaN,Ursus,NaN,Nearctic|Palearctic,Ursus_arctos
3,Orcinus_orca,3,killer+whale,Killer Whale,Orca,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,ARTIODACTYLA,WHIPPOMORPHA,CETACEA,ODONTOCETI,DELPHINOIDEA,DELPHINIDAE,INCERTAE SEDIS,NaN,Orcinus,NaN,Marine,Orcinus_orca
4,Castor_fiber,4,beaver,Eurasian Beaver,European Beaver,Mammalia,Placentalia,Boreoeutheria,Euarchontoglires,RODENTIA,SUPRAMYOMORPHA,CASTORIMORPHI,NaN,NaN,CASTORIDAE,NaN,NaN,Castor,NaN,Palearctic,Castor_fiber
5,Aplodontia_rufa,4,beaver,Mountain Beaver,Boomer|Sewellel|Point Arena Mountain Beaver|Po...,Mammalia,Placentalia,Boreoeutheria,Euarchontoglires,RODENTIA,SCIUROMORPHA,NaN,NaN,NaN,APLODONTIIDAE,NaN,NaN,Aplodontia,NaN,Nearctic,Aplodontia_rufa
6,Castor_canadensis,4,beaver,North American Beaver,American Beaver|Canadian Beaver,Mammalia,Placentalia,Boreoeutheria,Euarchontoglires,RODENTIA,SUPRAMYOMORPHA,CASTORIMORPHI,NaN,NaN,CASTORIDAE,NaN,NaN,Castor,NaN,Nearctic,Castor_canadensis
7,Canis_familiaris,5,dalmatian,Domestic Dog,Dog|Dingo|Feral Dog|New Guinea Singing Dog|Vil...,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,CARNIVORA,CANIFORMIA,CANOIDEA,NaN,NaN,CANIDAE,NaN,NaN,Canis,NaN,Domesticated,Canis_familiaris
8,Felis_catus,6,persian+cat,Domestic Cat,Cat,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,CARNIVORA,FELIFORMIA,FELOIDEA,NaN,NaN,FELIDAE,FELINAE,NaN,Felis,NaN,Domesticated,Felis_catus
9,Equus_caballus,7,horse,Domestic Horse,Horse|Feral Horse,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,PERISSODACTYLA,HIPOMORPHA,NaN,NaN,NaN,EQUIDAE,NaN,NaN,Equus,NaN,Domesticated,Equus_caballus


In [3]:
classes_list = ["Antelope", "Antelope", "Grizzly bear","Killer Whale","Beaver", "Beaver", "Beaver", "Dalmatian (dog)", "Persian cat", "Horse", "German Shepherd", 
               "Blue whale", "Siamese cat", "Skunk", "Mole (animal)", "Tiger", "Hippopotamus", "Leopard", "Moose", 
               "Spider monkey", "Humpback whale", "Elephant", "Gorilla", "Ox", "Ox", "Fox", "Sheep", "Seal", "Chimpanzee", 
                "Hamster", "Squirrel", "Rhinoceros","Rabbit", "Bat", "Giraffe", "Wolf", "Chihuahua (dog)",
               "Rat", "Rat", "Weasel", "Otter", "Buffalo", "Zebra", "Giant panda","Deer", "Bobcat", "Pig", "Lion", "Mouse",
                "Polar bear", "Collie", "Walrus", "Raccoon", "Cattle", "Dolphin"]

taxonomy_df['corrected_class_names'] = classes_list
taxonomy_df.head()

,sciName,animal_label,animal_class,mainCommonName,otherCommonNames,class,infraclass,magnorder,superorder,order,suborder,infraorder,parvorder,superfamily,family,subfamily,tribe,genus,subgenus,biogeographicRealm,CMW_sciName,corrected_class_names
0,Antilope_cervicapra,1,antelope,Blackbuck,Indian Antelope,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,ARTIODACTYLA,RUMINANTIA,PECORA,NaN,NaN,BOVIDAE,ANTILOPINAE,ANTILOPINI,Antilope,NaN,Palearctic,Antilope_cervicapra,Antelope
1,Pantholops_hodgsonii,1,antelope,Chiru,Tibetan Antelope,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,ARTIODACTYLA,RUMINANTIA,PECORA,NaN,NaN,BOVIDAE,ANTILOPINAE,CAPRINI,Pantholops,NaN,Palearctic,Pantholops_hodgsonii,Antelope
2,Ursus_arctos,2,grizzly+bear,Brown Bear,Grizzly Bear|Kodiak Bear,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,CARNIVORA,CANIFORMIA,ARCTOIDEA,NaN,URSOIDEA,URSIDAE,URSINAE,NaN,Ursus,NaN,Nearctic|Palearctic,Ursus_arctos,Grizzly bear
3,Orcinus_orca,3,killer+whale,Killer Whale,Orca,Mammalia,Placentalia,Boreoeutheria,Laurasiatheria,ARTIODACTYLA,WHIPPOMORPHA,CETACEA,ODONTOCETI,DELPHINOIDEA,DELPHINIDAE,INCERTAE SEDIS,NaN,Orcinus,NaN,Marine,Orcinus_orca,Killer Whale
4,Castor_fiber,4,beaver,Eurasian Beaver,European Beaver,Mammalia,Placentalia,Boreoeutheria,Euarchontoglires,RODENTIA,SUPRAMYOMORPHA,CASTORIMORPHI,NaN,NaN,CASTORIDAE,NaN,NaN,Castor,NaN,Palearctic,Castor_fiber,Beaver


# 3. URI generation & RDF generation

### 3.1 Set up dictionary to hold the URI's
Adapted from N3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5 solution

In [4]:
#Dictionary that keeps the URIs. Specially useful if accessing a remote service to get a candidate URI to avoid repeated calls
stringToURI = dict()

## 3.2 Set up empty graph and Namespace

In [5]:
#Create empty graph
g = Graph()

#Define Namespace
taxonomy_onto_uri = "http://www.semanticweb.org/inm363/AnimalTaxonomy#"
      
at = Namespace(taxonomy_onto_uri)
dbo = Namespace("http://dbpedia.org/ontology/") 
dbr = Namespace("http://dbpedia.org/resource/") 
owl = Namespace("http://www.w3.org/2002/07/owl#") 

#Bind Prefixes 
g.bind("at", at) 
g.bind("dbo", dbo) 
g.bind("dbr", dbr) 
g.bind("owl", owl) 

## 3.3 Define Code to generate Triples
Reused or Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

### 3.3.1 Modular Mapping functions 

In [6]:
# Reused or Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

def is_nan(x):
        return (x != x)         
            
        
def createURIForEntity(name, useExternalURI):
    stringToURI[name] = at + name.replace(" ", "_")
    
    if useExternalURI is True: #We connect to online KG
        uri = getExternalKGURI(name)
        if uri!="":
            stringToURI[name]=uri
    
    return stringToURI[name]

  
def getExternalKGURI(name):
    entities = DBpediaLookup().getKGEntities(name, 2)
    current_sim = -1
    current_uri=''
    for ent in entities:           
        isub_score = isub(name, ent.label) 
        if current_sim < isub_score:
            current_uri = ent.ident
            current_sim = isub_score    
        
    return current_uri 

    
def mappingToCreateTypeTriple(dataframe, subject_column, class_column, useExternalURI):  
    for subject, classname in zip(dataframe[subject_column], dataframe[class_column]):
        if isinstance(classname, str) is False:
            pass
        
        else: 
            if subject.lower() in stringToURI:
                subject_uri=stringToURI[subject]
            else:
                subject_uri=createURIForEntity(subject, useExternalURI)    
            
            
            class_uri=stringToURI[classname.capitalize()]
            
            #New triple
            g.add((URIRef(subject_uri), RDF.type, URIRef(class_uri)))
            print('\nExample generated object property triple for',URIRef(subject_uri), RDF.type, URIRef(class_uri))
   

          
def mappingToCreateSubClassOfTriple(dataframe, subclass, mainclass, useExternalURI):
    #Get subclass types and generate URI
    subclass_df = dataframe.copy()
    subclass_df.dropna(subset=[subclass],inplace = True)
    subclass_types = subclass_df[subclass].unique()

    subclass_dict = {}

    for i in subclass_types:
        subclass_dict[i] = createURIForEntity(i.capitalize(), useExternalURI)
        
    # Print dictionary items 
    print('\033[1m',subclass.capitalize(), 'Classes and URIs','\033[0m')
    for k, v in subclass_dict.items():
        print(k, v)   
    
    print('\033[1m','\nGenrated Triples','\033[0m')
    #Create SubClassOf Triples
    for k, v in subclass_dict.items():    
        for subclasses, mainclasses, in zip(subclass_df[subclass], subclass_df[mainclass]):
            if k in subclasses:
                if str(mainclasses.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(mainclasses.capitalize())]
                else:
                    entity_uri=createURIForEntity(mainclasses.capitalize(), useExternalURI)    

                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
        
        #Print triples
        print('\n',v, RDFS.subClassOf, URIRef(entity_uri))
    

def mappingToCreateLiteralTriple(dataframe, subject_column, object_column, predicate, datatype):
        
        for subject, lit_value in zip(dataframe[subject_column], dataframe[object_column]):
            
            if is_nan(lit_value) or lit_value==None or lit_value=="":
                pass
            
            else:
                #Uri as already created
                entity_uri = stringToURI[subject.lower()]
                    
                #Literal
                lit = Literal(lit_value, datatype=datatype)
                
                #New triple
                g.add((URIRef(entity_uri), predicate, lit))                     
                                   
            print('\033[1m','\nExample generated data property triple for',subject_column,'and',object_column,':','\033[0m', URIRef(entity_uri), predicate, lit)
            print('')
                         
            
def mappingToCreateObjectTriple(dataframe, subject_column, object_name, predicate):
        
    for subject in dataframe[subject_column]:
            
        if is_nan(object):
            pass
            
        else:
            #Uri as already created
            subject_uri = stringToURI[subject.lower()]
            object_uri = object_name
                    
            #New triple
            g.add((URIRef(subject_uri), predicate, URIRef(object_uri)))
        print('\n','\033[1m','Example generated object property triple for','\033[0m',URIRef(subject_uri), predicate, URIRef(object_uri))
        print('')
        

# 4 Define SubClassOf Type Triples
Adapted from IN3067/INM713 Semantic Web Technologies and Knowledge Graphs Lab 5

### 4.1 Empty dictionary

In [7]:
stringToURI.clear() #empty dict 

### 4.2 Animal taxonomy

### 4.2.1 Mapping Infaclass 

In [8]:
mappingToCreateSubClassOfTriple(taxonomy_df, 'infraclass', 'class', True)

 Infraclass Classes and URIs 
Placentalia http://dbpedia.org/resource/Placentalia
 
Genrated Triples 

 http://dbpedia.org/resource/Placentalia http://www.w3.org/2000/01/rdf-schema#subClassOf http://dbpedia.org/resource/Mammal


### 4.2.2 Mapping MagnOrder

In [9]:
mappingToCreateSubClassOfTriple(taxonomy_df, 'magnorder', 'infraclass', True)

 Magnorder Classes and URIs 
Boreoeutheria http://dbpedia.org/resource/Boreoeutheria
Atlantogenata http://dbpedia.org/resource/Atlantogenata
 
Genrated Triples 

 http://dbpedia.org/resource/Boreoeutheria http://www.w3.org/2000/01/rdf-schema#subClassOf http://dbpedia.org/resource/Placentalia

 http://dbpedia.org/resource/Atlantogenata http://www.w3.org/2000/01/rdf-schema#subClassOf http://dbpedia.org/resource/Placentalia


### 4.2.4 Mapping SuperOrder

In [10]:
mappingToCreateSubClassOfTriple(taxonomy_df, 'superorder' ,'magnorder', True)

 Superorder Classes and URIs 
Laurasiatheria http://dbpedia.org/resource/Laurasiatheria
Euarchontoglires http://dbpedia.org/resource/Euarchontoglires
Afrotheria http://dbpedia.org/resource/Afrotheria
 
Genrated Triples 

 http://dbpedia.org/resource/Laurasiatheria http://www.w3.org/2000/01/rdf-schema#subClassOf http://dbpedia.org/resource/Boreoeutheria

 http://dbpedia.org/resource/Euarchontoglires http://www.w3.org/2000/01/rdf-schema#subClassOf http://dbpedia.org/resource/Boreoeutheria

 http://dbpedia.org/resource/Afrotheria http://www.w3.org/2000/01/rdf-schema#subClassOf http://dbpedia.org/resource/Atlantogenata


### 4.2.5 Mapping Order

In [11]:
mappingToCreateSubClassOfTriple(taxonomy_df, 'order' ,'superorder', False)

 Order Classes and URIs 
ARTIODACTYLA http://www.semanticweb.org/inm363/AnimalTaxonomy#Artiodactyla
CARNIVORA http://www.semanticweb.org/inm363/AnimalTaxonomy#Carnivora
RODENTIA http://www.semanticweb.org/inm363/AnimalTaxonomy#Rodentia
PERISSODACTYLA http://www.semanticweb.org/inm363/AnimalTaxonomy#Perissodactyla
EULIPOTYPHLA http://www.semanticweb.org/inm363/AnimalTaxonomy#Eulipotyphla
PRIMATES http://www.semanticweb.org/inm363/AnimalTaxonomy#Primates
PROBOSCIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Proboscidea
LAGOMORPHA http://www.semanticweb.org/inm363/AnimalTaxonomy#Lagomorpha
CHIROPTERA http://www.semanticweb.org/inm363/AnimalTaxonomy#Chiroptera
 
Genrated Triples 

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Artiodactyla http://www.w3.org/2000/01/rdf-schema#subClassOf http://dbpedia.org/resource/Laurasiatheria

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Carnivora http://www.w3.org/2000/01/rdf-schema#subClassOf http://dbpedia.org/resource/Laurasiatheria



### 4.2.5 Mapping SubOrder

In [12]:
mappingToCreateSubClassOfTriple(taxonomy_df, 'suborder' ,'order', False)

 Suborder Classes and URIs 
RUMINANTIA http://www.semanticweb.org/inm363/AnimalTaxonomy#Ruminantia
CANIFORMIA http://www.semanticweb.org/inm363/AnimalTaxonomy#Caniformia
WHIPPOMORPHA http://www.semanticweb.org/inm363/AnimalTaxonomy#Whippomorpha
SUPRAMYOMORPHA http://www.semanticweb.org/inm363/AnimalTaxonomy#Supramyomorpha
SCIUROMORPHA http://www.semanticweb.org/inm363/AnimalTaxonomy#Sciuromorpha
FELIFORMIA http://www.semanticweb.org/inm363/AnimalTaxonomy#Feliformia
HIPOMORPHA http://www.semanticweb.org/inm363/AnimalTaxonomy#Hipomorpha
HAPLORHINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Haplorhini
CERATOMORPHA http://www.semanticweb.org/inm363/AnimalTaxonomy#Ceratomorpha
VESPERTILIONIFORMES http://www.semanticweb.org/inm363/AnimalTaxonomy#Vespertilioniformes
SUINA http://www.semanticweb.org/inm363/AnimalTaxonomy#Suina
 
Genrated Triples 

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Ruminantia http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363

### 4.2.6 Mapping Infraorder

In [13]:
mappingToCreateSubClassOfTriple(taxonomy_df, 'infraorder' ,'suborder', False)

 Infraorder Classes and URIs 
PECORA http://www.semanticweb.org/inm363/AnimalTaxonomy#Pecora
ARCTOIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Arctoidea
CETACEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Cetacea
CASTORIMORPHI http://www.semanticweb.org/inm363/AnimalTaxonomy#Castorimorphi
CANOIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Canoidea
FELOIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Feloidea
ANCODONTA http://www.semanticweb.org/inm363/AnimalTaxonomy#Ancodonta
SIMIIFORMES http://www.semanticweb.org/inm363/AnimalTaxonomy#Simiiformes
MYOMORPHI http://www.semanticweb.org/inm363/AnimalTaxonomy#Myomorphi
 
Genrated Triples 

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Pecora http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Ruminantia

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Arctoidea http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Canifor

### 4.2.7 Mapping Parvorder

In [14]:
mappingToCreateSubClassOfTriple(taxonomy_df,'parvorder', 'infraorder', False)

 Parvorder Classes and URIs 
ODONTOCETI http://www.semanticweb.org/inm363/AnimalTaxonomy#Odontoceti
MYSTICETI http://www.semanticweb.org/inm363/AnimalTaxonomy#Mysticeti
PLATYRRHINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Platyrrhini
CATARRHINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Catarrhini
 
Genrated Triples 

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Odontoceti http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Cetacea

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Mysticeti http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Cetacea

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Platyrrhini http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Simiiformes

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Catarrhini http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Sim

### 4.2.8 Mapping Superfamily

In [15]:
subclass = 'superfamily'

subclass_df = taxonomy_df.copy()
subclass_df.dropna(subset=[subclass],inplace = True)
subclass_types = subclass_df[subclass].unique()

subclass_dict = {}

for i in subclass_types:
    subclass_dict[i] = createURIForEntity(i.capitalize(), False) 

# Print dictionary items 
print('\033[1m',subclass.capitalize(), 'Classes and URIs','\033[0m')
for k, v in subclass_dict.items():
    print(k, v)   

 Superfamily Classes and URIs 
URSOIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursoidea
DELPHINOIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Delphinoidea
MUSTELOIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Musteloidea
HOMINOIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Hominoidea
PHOCOIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Phocoidea
MUROIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Muroidea
NOCTILIONOIDEA http://www.semanticweb.org/inm363/AnimalTaxonomy#Noctilionoidea


In [16]:
for k, v in subclass_dict.items():    
    for superfamily, parvorder, infraorder, suborder, order in zip(subclass_df[subclass], subclass_df['parvorder'], subclass_df['infraorder'], subclass_df['suborder'], subclass_df['order']):
        if k in superfamily:
            if isinstance(parvorder, str) is False and isinstance(infraorder, str) is False and isinstance(suborder, str) is False:                    
                if str(order.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(order.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(order.capitalize()), False)  
                    
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
              
            
            elif isinstance(parvorder, str) is False and isinstance(infraorder, str) is False:      
                if str(suborder.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(suborder.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(suborder.capitalize()), False)    
                   
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
              
                
            elif isinstance(parvorder, str) is False:      
                if str(infraorder.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(infraorder.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(infraorder.capitalize()), False)    
                   
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
                
                
            else:
                if str(parvorder.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(parvorder.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(parvorder.capitalize()), False)    
                   
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
        
    print('\n',URIRef(v), RDFS.subClassOf, URIRef(entity_uri))
                
  


 http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursoidea http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Arctoidea

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Delphinoidea http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Odontoceti

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Musteloidea http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Arctoidea

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Hominoidea http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Catarrhini

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Phocoidea http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Arctoidea

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Muroidea http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxon

### 4.2.9 Mapping Family

In [17]:
subclass = 'family'

subclass_df = taxonomy_df.copy()
subclass_df.dropna(subset=[subclass],inplace = True)
subclass_types = subclass_df[subclass].unique()

subclass_dict = {}

for i in subclass_types:
    subclass_dict[i] = createURIForEntity(i.capitalize(), False) 

# Print dictionary items 
print('\033[1m',subclass.capitalize(), 'Classes and URIs','\033[0m')
for k, v in subclass_dict.items():
    print(k, v)   



 Family Classes and URIs 
BOVIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Bovidae
URSIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursidae
DELPHINIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Delphinidae
CASTORIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Castoridae
APLODONTIIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Aplodontiidae
CANIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Canidae
FELIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Felidae
EQUIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Equidae
BALAENOPTERIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Balaenopteridae
MEPHITIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Mephitidae
TALPIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Talpidae
HIPPOPOTAMIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Hippopotamidae
CERVIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Cervidae
ATELIDAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Ateli

In [18]:
for k, v in subclass_dict.items():    
    for family, superfamily, parvorder, infraorder, suborder, order in zip(subclass_df['family'], 
                                                                           subclass_df['superfamily'], 
                                                                           subclass_df['parvorder'], 
                                                                           subclass_df['infraorder'], 
                                                                           subclass_df['suborder'], 
                                                                           subclass_df['order']):
        if k in family:
            if isinstance(superfamily, str) is False and isinstance(parvorder, str) is False and isinstance(infraorder, str) is False and isinstance(suborder, str) is False:                    
                if str(order.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(order.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(order.capitalize()), False)  
                    
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
               
            elif isinstance(superfamily, str) is False and isinstance(parvorder, str) is False and isinstance(infraorder, str) is False:      
                if str(suborder.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(suborder.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(suborder.capitalize()), False)    
                   
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
 
            elif isinstance(superfamily, str) is False and isinstance(parvorder, str) is False:      
                if str(infraorder.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(infraorder.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(infraorder.capitalize()), False)    
                   
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
               
                
            elif isinstance(superfamily, str) is False:
                if str(parvorder.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(parvorder.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(parvorder.capitalize()), False)    
                   
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
              
                
            else:
                if str(superfamily.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(superfamily.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(superfamily.capitalize()), False)    
                   
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
            
                
    print('\n',v, RDFS.subClassOf, URIRef(entity_uri))


 http://www.semanticweb.org/inm363/AnimalTaxonomy#Bovidae http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Pecora

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursidae http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursoidea

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Delphinidae http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Delphinoidea

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Castoridae http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Castorimorphi

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Aplodontiidae http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Sciuromorpha

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Canidae http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxo

### 4.2.10 Mapping Subfamily

In [19]:
mappingToCreateSubClassOfTriple(taxonomy_df,'subfamily', 'family', False)

 Subfamily Classes and URIs 
ANTILOPINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilopinae
URSINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursinae
INCERTAE SEDIS http://www.semanticweb.org/inm363/AnimalTaxonomy#Incertae_sedis
FELINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Felinae
MEPHITINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Mephitinae
TALPINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Talpinae
PANTHERINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Pantherinae
CAPREOLINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Capreolinae
ATELINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Atelinae
HOMININAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Homininae
BOVINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Bovinae
CRICETINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Cricetinae
SCIURINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#Sciurinae
DESMODONTINAE http://www.semanticweb.org/inm363/AnimalTaxonomy#D

### 4.2.11 Mapping Tribe

In [20]:
subclass = 'tribe'

subclass_df = taxonomy_df.copy()
subclass_df.dropna(subset=[subclass],inplace = True)
subclass_types = subclass_df[subclass].unique()

subclass_dict = {}

for i in subclass_types:
    subclass_dict[i] = createURIForEntity(i.capitalize(), False) 

# Print dictionary items 
print('\033[1m',subclass.capitalize(), 'Classes and URIs','\033[0m')
for k, v in subclass_dict.items():
    print(k, v)   


 Tribe Classes and URIs 
ANTILOPINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilopini
CAPRINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Caprini
TALPINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Talpini
ALCEINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Alceini
BOVINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Bovini
SCIURINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Sciurini
DESMODONTINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Desmodontini
ICHTHYOMYINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Ichthyomyini
HYDROMYINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Hydromyini
CAPREOLINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Capreolini
WIEDOMYINI http://www.semanticweb.org/inm363/AnimalTaxonomy#Wiedomyini


In [21]:
for k, v in subclass_dict.items():    
    for tribe, subfamily, family in zip(subclass_df[subclass], subclass_df['subfamily'], subclass_df['family']):
        if k in tribe:
            if isinstance(subfamily, str) is False: 
                if str(family.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(family.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(family.capitalize()), False)  
                    
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
              
            else:
                if str(subfamily.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(subfamily.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(subfamily.capitalize()), False)    
                   
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
        
    print('\n',URIRef(v), RDFS.subClassOf, URIRef(entity_uri))
                


 http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilopini http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilopinae

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Caprini http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilopinae

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Talpini http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Talpinae

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Alceini http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Capreolinae

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Bovini http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Bovinae

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Sciurini http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Sciurina

### 4.2.12 Mapping Genus

In [22]:
subclass = 'genus'

subclass_df = taxonomy_df.copy()
subclass_df.dropna(subset=[subclass],inplace = True)
subclass_types = subclass_df[subclass].unique()

subclass_dict = {}

for i in subclass_types:
    subclass_dict[i] = createURIForEntity(i.capitalize(), False) 

# Print dictionary items 
print('\033[1m',subclass.capitalize(), 'Classes and URIs','\033[0m')
for k, v in subclass_dict.items():
    print(k, v)   


 Genus Classes and URIs 
Antilope http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilope
Pantholops http://www.semanticweb.org/inm363/AnimalTaxonomy#Pantholops
Ursus http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursus
Orcinus http://www.semanticweb.org/inm363/AnimalTaxonomy#Orcinus
Castor http://www.semanticweb.org/inm363/AnimalTaxonomy#Castor
Aplodontia http://www.semanticweb.org/inm363/AnimalTaxonomy#Aplodontia
Canis http://www.semanticweb.org/inm363/AnimalTaxonomy#Canis
Felis http://www.semanticweb.org/inm363/AnimalTaxonomy#Felis
Equus http://www.semanticweb.org/inm363/AnimalTaxonomy#Equus
Balaenoptera http://www.semanticweb.org/inm363/AnimalTaxonomy#Balaenoptera
Spilogale http://www.semanticweb.org/inm363/AnimalTaxonomy#Spilogale
Talpa http://www.semanticweb.org/inm363/AnimalTaxonomy#Talpa
Panthera http://www.semanticweb.org/inm363/AnimalTaxonomy#Panthera
Hippopotamus http://www.semanticweb.org/inm363/AnimalTaxonomy#Hippopotamus
Alces http://www.semanticweb.org/inm363/AnimalT

In [23]:
for k, v in subclass_dict.items():    
    for genus, tribe, subfamily, family in zip(subclass_df[subclass], subclass_df['tribe'],  subclass_df['subfamily'], subclass_df['family']):
        if k == genus:
            if isinstance(tribe, str) is False and isinstance(subfamily, str) is False: 
                if str(family.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(family.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(family.capitalize()), False)  
                    
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
                
            elif isinstance(tribe, str) is False: 
                if str(subfamily.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(subfamily.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(subfamily.capitalize()), False)  
                    
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
              
            else:
                if str(tribe.capitalize()) in stringToURI:
                    entity_uri=stringToURI[str(tribe.capitalize())]
                else:
                    entity_uri=createURIForEntity(str(tribe.capitalize()), False)    
                   
                g.add((URIRef(v), RDFS.subClassOf, URIRef(entity_uri)))
        
    print('\n',URIRef(v), RDFS.subClassOf, URIRef(entity_uri))
                


 http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilope http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilopini

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Pantholops http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Caprini

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursus http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursinae

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Orcinus http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Incertae_sedis

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Castor http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Castoridae

 http://www.semanticweb.org/inm363/AnimalTaxonomy#Aplodontia http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.semanticweb.org/inm363/AnimalTaxonomy#Aplodon

# 5. Mapping animal individuals to Animal Taxonomy

### 5.1 Class

In [24]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'class', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Mammal

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Mammal

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Mammal

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Mammal

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Mammal

Example generated object property triple for http://www.se

### 5.2 Infraclass

In [25]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'infraclass', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Placentalia

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Placentalia

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Placentalia

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Placentalia

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Placentalia

Example generated object property

### 5.3 Magnorder

In [26]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'magnorder', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Boreoeutheria

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Boreoeutheria

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Boreoeutheria

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Boreoeutheria

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Boreoeutheria

Example generated objec

### 5.4 Superorder

In [27]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'superorder', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Laurasiatheria

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Laurasiatheria

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Laurasiatheria

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Laurasiatheria

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/resource/Euarchontoglires

Example generate

### 5.5 Order

In [28]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'order', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Artiodactyla

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Artiodactyla

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Carnivora

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Artiodactyla

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.w3.org/1999/02/22-rdf-synt

### 5.6 Suborder

In [29]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'suborder', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Ruminantia

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Ruminantia

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Caniformia

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Whippomorpha

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.w3.org/1999/02/22-rdf-syntax-

### 5.7 Infraorder

In [30]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'infraorder', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Pecora

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Pecora

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Arctoidea

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Cetacea

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.w3.org/1999/02/22-rdf-syntax-ns#type http:/

### 5.9 Parvorder

In [31]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'parvorder', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Odontoceti

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#blue+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Mysticeti

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#spider+monkey http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Platyrrhini

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#humpback+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Mysticeti

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#gorilla http://www.w3.org/1999/02/22-rdf-

### 5.9 Superfamily

In [32]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'superfamily', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursoidea

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Delphinoidea

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#skunk http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Musteloidea

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#gorilla http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Hominoidea

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#seal http://www.w3.org/1999/02/22-rdf-syntax-ns#type

### 5.9 Family

In [33]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'family', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Bovidae

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Bovidae

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursidae

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Delphinidae

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.w3.org/1999/02/22-rdf-syntax-ns#type ht

### 5.10 Subfamily

In [34]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'subfamily', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilopinae

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilopinae

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursinae

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Incertae_sedis

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#persian+cat http://www.w3.org/1999/02/22-rdf-s

### 5.11 Tribe

In [35]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'tribe', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilopini

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Caprini

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#mole http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Talpini

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#moose http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Alceini

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#ox http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb

### 5.12 Genus

In [36]:
mappingToCreateTypeTriple(taxonomy_df,'animal_class', 'genus', False) 


Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Antilope

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Pantholops

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Ursus

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.semanticweb.org/inm363/AnimalTaxonomy#Orcinus

Example generated object property triple for http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.w3.org/1999/02/22-rdf-syntax-ns#type http

# 6. Create Litral type Triples

In [37]:
mappingToCreateLiteralTriple(taxonomy_df, 'animal_class', 'corrected_class_names', at.animalNames, XSD.string)

 
Example generated data property triple for animal_class and corrected_class_names :  http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.semanticweb.org/inm363/AnimalTaxonomy#animalNames Antelope

 
Example generated data property triple for animal_class and corrected_class_names :  http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.semanticweb.org/inm363/AnimalTaxonomy#animalNames Antelope

 
Example generated data property triple for animal_class and corrected_class_names :  http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.semanticweb.org/inm363/AnimalTaxonomy#animalNames Grizzly bear

 
Example generated data property triple for animal_class and corrected_class_names :  http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.semanticweb.org/inm363/AnimalTaxonomy#animalNames Killer Whale

 
Example generated data property triple for animal_class and corrected_class_names :  http://www.semanticweb.org/inm363

In [38]:
mappingToCreateLiteralTriple(taxonomy_df, 'animal_class', 'sciName', at.sciName, XSD.string)

 
Example generated data property triple for animal_class and sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.semanticweb.org/inm363/AnimalTaxonomy#sciName Antilope_cervicapra

 
Example generated data property triple for animal_class and sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.semanticweb.org/inm363/AnimalTaxonomy#sciName Pantholops_hodgsonii

 
Example generated data property triple for animal_class and sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.semanticweb.org/inm363/AnimalTaxonomy#sciName Ursus_arctos

 
Example generated data property triple for animal_class and sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.semanticweb.org/inm363/AnimalTaxonomy#sciName Orcinus_orca

 
Example generated data property triple for animal_class and sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http://www.semanticweb.org/inm363/Animal

In [39]:
mappingToCreateLiteralTriple(taxonomy_df, 'animal_class', 'CMW_sciName', at.CMW_sciName, XSD.string)

 
Example generated data property triple for animal_class and CMW_sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.semanticweb.org/inm363/AnimalTaxonomy#CMW_sciName Antilope_cervicapra

 
Example generated data property triple for animal_class and CMW_sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope http://www.semanticweb.org/inm363/AnimalTaxonomy#CMW_sciName Pantholops_hodgsonii

 
Example generated data property triple for animal_class and CMW_sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#grizzly+bear http://www.semanticweb.org/inm363/AnimalTaxonomy#CMW_sciName Ursus_arctos

 
Example generated data property triple for animal_class and CMW_sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#killer+whale http://www.semanticweb.org/inm363/AnimalTaxonomy#CMW_sciName Orcinus_orca

 
Example generated data property triple for animal_class and CMW_sciName :  http://www.semanticweb.org/inm363/AnimalTaxonomy#beaver http

## 7. Print all triples in the  generated RDF data

In [40]:
print("The graph contains '" + str(len(g)) + "' triples.")
#for subj, pred, obj in g:
    #print(subj, pred, obj)

The graph contains '823' triples.


# 8. Save the Generated RDF data 

In [41]:
g.serialize(destination='AT_KG02_Animal_Taxonomy.ttl', format='ttl') 
g.serialize(destination='C:/Users/Abhimanyu Acharya/Documents/GitHub/Dissertation/OWL2Vec-Star-master/dissertation/AT_KG02_Animal_Taxonomy.owl', format='xml') 


# 9. Run some queries to check the generated RDF data

In [42]:
g_test = Graph()

#Load Generated RDF
Generated_rdf = "AT_KG02_Animal_Taxonomy.ttl" # define path to Generated RDF
g_test.load(Generated_rdf ,  format=guess_format(Generated_rdf)) #load Generated RDF

In [43]:
qres = g_test.query("""
                SELECT DISTINCT ?animals WHERE {
                ?animals rdf:type at:Pantholops .  
                }    
            """)


print("%s animals satisfying the query." % (str(len(qres))))

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    line_str = row.animals
    print(line_str)

1 animals satisfying the query.
http://www.semanticweb.org/inm363/AnimalTaxonomy#antelope
